In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
#from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),   #二维卷积核，self.conv2d = nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
            #stride:卷积核在图像窗口上每次平移的间隔，即所谓的步长
            #Padding即所谓的图像填充，后面的int型常数代表填充的多少（行数、列数），默认为0。
            # 需要注意的是这里的填充包括图像的上下左右，以padding = 1为例，若原始图像大小为32x32，那么padding后的图像大小就变成了34x34，而不是33x33    
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),   #torch.nn.Flatten()默认从第二维开始平坦化
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=64)


In [ ]:
model = Model().cuda()  #调用model.cuda()，可以将模型加载到GPU上去。
#添加tensorboard可视化数据
#writer = SummaryWriter('../logs_tensorboard')  #用于在给定目录中创建事件文件，并向其中添加摘要和事件
loss = nn.CrossEntropyLoss().cuda()
optimizer  = torch.optim.SGD(model.parameters(),lr=0.003)
#i = 1 # 用于绘制测试集的tensorboard


In [ ]:
Train_correction = []
Test_correction = []
Loss = []

In [ ]:
for epoch in range(50):
    if epoch == 40:
        for p in optimizer.param_groups:
            p['lr'] = 0.001
    model.train() # 也可以不写，规范的话是写，用来表明训练步骤
    correct = 0
    total = 0
    loss_ = 0.0
    for data in train_dataloader:
        # 数据分开 一个是图片数据，一个是真实值
        imgs,targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        # 拿到预测值
        output = model(imgs)
        # 计算损失值
        loss_in = loss(output,targets)
        # 优化开始~ ~ 先梯度清零
        optimizer.zero_grad()
        # 反向传播+更新
        loss_in.backward()
        optimizer.step()
        loss_ += loss_in.item()
        _,predicted = torch.max(output.data, 1)        #torch.max(input, dim),dim是max函数索引的维度0/1，0是每列的最大值，1是每行的最大值
        #函数会返回两个tensor，第一个tensor是每行的最大值，softmax的输出中最大的是1，所以第一个tensor是全1的tensor；第二个tensor是每行最大值的索引。
        total += targets.size(0)     #计算lables总量        
        #torrch.size(0)中的0表示第0维度的数据数量,如a = torch.tensor([[1,2,3], [4,5,6]])，a.size（0）为2，即有[1,2,3]，[4,5,6]两个数据
        correct += (predicted == targets).sum().item()
        #(a == b).sum()，若a,b为张量，返回tensor(1)。若为numpy，则返回1。.item()用于取出tensor中的值。
    a = float(100 * correct / total)
    Train_correction.append(a) 
    Loss.append(loss_)
    # 每轮训练完成跑一下测试数据看看情况
    accurate = 0
    total_ = 0
    model.eval() # 也可以不写，规范的话就写，用来表明是测试步骤
    with torch.no_grad():
        for data in test_dataloader:
            # 这里的每一次循环 都是一个minibatch  一次for循环里面有64个数据。
            imgs , targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            output = model(imgs)
            _,predicted_ = torch.max(output.data, 1)
            total_ += targets.size(0)
            accurate += (predicted_ == targets).sum().item()
    b = float(100 * accurate / total_)
    Test_correction.append(b) 

In [ ]:
x = [i for i in range(1,51)]
print(len(x))
plt.figure(figsize=(15,15))
plt.subplot(3,3,1)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('train_correct')
plt.plot(x,Train_correction)
plt.subplot(3,3,2)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('test_correct')
plt.plot(x,Test_correction)
plt.subplot(3,3,3)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.plot(x,Loss)
plt.show()
print(x)
print(len(Train_correction))
print(len(Test_correction))